PREGUNTAR: Puedo chorearme mi archivo punto2/contabilizado/output.txt con estructura

```sql 
id_retador puntos_total cantidad
```

para simplificar el calculo de la PP?

Primer JOB es solo para formatear el DATASET COMO UN INNER JOIN 

In [ ]:
from MRE import Job
inputDir = "./jugadores/"
promedioDir = "../punto2/contabilizado/"
outputDir = "./format/"

In [ ]:
def fmap(key, value, context):
    lista = value.split()
    #print(key)
    context.write((int(lista[0]),'RETADO'), int(key))

def fmap2(key, value, context):
    lista = value.split()
    #print("2 " + key)

    context.write((int(key), 'PP')  , (int(lista[0])/int(lista[1])))

def cmpShort(key1,key2):
    if(key1[0] == key2[0]):
        return 0
    elif(key1[0] < key2[0]):
        return -1
    else:
        return 1

def cmpShuffle(key1, key2):
    if(key1[1] == key2[1]):
        return 0
    elif(key1[1] == "PP"): #PP es el primero 
        return -1
    else:
        return 1 #Caso retado

def fred(key, values, context):
    acumular_retados = set()
    pp = next(values)  
    for v in values:
        acumular_retados.add(v)
    context.write(key[0], (pp,acumular_retados,puntaje_heroico_inicial))

    
puntaje_heroico_inicial = 1

jobCount = Job(inputDir, outputDir, fmap, fred)
jobCount.setSortCmp(cmpShuffle)
jobCount.setShuffleCmp(cmpShort)
jobCount.setParams(puntaje_heroico_inicial)
jobCount.addInputPath(promedioDir,fmap2)

success = jobCount.waitForCompletion()

print(success)

Segundo job iterativo

In [ ]:
inputDir = "./format/"
outputDir = "./out/"

In [ ]:
def fmap(key, value, context):
    lista = value.split()
    context.write(key, ("ESTRUCTURA", value))
    for v in value:
        context.write(key, ("CONTRIBUCION", lista[1],lista[3]))

def cmpShort(key1,key2):
    if(key1[0] == key2[0]):
        return 0
    elif(key1[0] < key2[0]):
        return -1
    else:
        return 1

def cmpShuffle(key1, key2):
    if(key1[1] == key2[1]):
        return 0
    elif(key1[1] == "ESTRUCTURA"): #PP es el primero 
        return -1
    else:
        return 1 #Caso retado

def fred(key, values, context):
    valor = next(values)
    actual = (key,valor)
    PPi = valor[0]
    acumulador=0
    for v in values: 
        PHv = v[2]
        PPv = v[1]
        acumulador= PHv * (PPi/PPv)
    acumulador = alfa*acumulador + (1-alfa)
    valor = (valor[0],valor[1], valor[2], acumulador)
    context.write(key,valor)

alfa = 0.1
jobCountBoth = Job(inputDir, outputDir, fmap, fred)
jobCount.setParams(alfa)
success = jobCountBoth.waitForCompletion()

print(success)

Tercer Job saca top 10 